In [1]:
# 📚 Libraries 
import kagglehub
import pandas as pd
import numpy as np
import os

# 📊 Visualizations
import matplotlib.pyplot as plt
import seaborn as sns
import plotly.express as px
import plotly.graph_objects as g

# 🤖 Machine Learning
from sklearn.model_selection import train_test_split
from sklearn.linear_model import LinearRegression
from sklearn.metrics import r2_score, mean_squared_error, mean_absolute_error 

**Goals**

- Create a time series regression model to predict S&P value and/or stock prices. 

- Identify high and low performance stocks amongs the list. 

- Comparioso If you invested $1000 in S&P vs pick 2/3 stocks. 

In [2]:
# Download latest version
path = kagglehub.dataset_download("andrewmvd/sp-500-stocks")


In [3]:
# Print all files in the dataset path
print(os.listdir(path))

['sp500_stocks.csv', 'sp500_companies.csv', 'sp500_index.csv']


In [4]:
csv_file_path = os.path.join(path, 'sp500_stocks.csv')
csv_file_path2 = os.path.join(path, 'sp500_companies.csv')
csv_file_path3 = os.path.join(path, 'sp500_index.csv')
data = pd.read_csv(csv_file_path)
df = pd.read_csv(csv_file_path2)
sp = pd.read_csv(csv_file_path3)

In [5]:
data.columns = [col.lower().replace(" ", "_")for col in data.columns] #snake_case
df.columns = [col.lower().replace(" ", "_")for col in df.columns] #snake_case
sp.columns = [col.lower().replace(" ", "_")for col in sp.columns] #snake_case

In [6]:
data2 = data.copy()

In [7]:
data2.dtypes

date          object
symbol        object
adj_close    float64
close        float64
high         float64
low          float64
open         float64
volume       float64
dtype: object

In [8]:
# Delete Columns 
data2.drop(columns=['high', 'low', 'open','close'], inplace=True)

In [9]:
data2.isna().sum()

date              0
symbol            0
adj_close    101626
volume       101626
dtype: int64

In [10]:
data2.dropna(how='any', inplace=True)

In [11]:
data2

,date,symbol,adj_close,volume
0,2010-01-04,MMM,43.783867,3640265.0
1,2010-01-05,MMM,43.509628,3405012.0
2,2010-01-06,MMM,44.126682,6301126.0
3,2010-01-07,MMM,44.158325,5346240.0
4,2010-01-08,MMM,44.469463,4073337.0
...,...,...,...,...
1890269,2024-12-02,ZTS,176.809998,2391500.0
1890270,2024-12-03,ZTS,176.940002,2679000.0
1890271,2024-12-04,ZTS,175.320007,2687000.0
1890272,2024-12-05,ZTS,174.770004,2442000.0


In [12]:
data2['date'] = pd.to_datetime(data2['date'])

In [13]:
data2.dtypes

date         datetime64[ns]
symbol               object
adj_close           float64
volume              float64
dtype: object

In [14]:
data2['year'] = data2['date'].dt.year
data2['month'] = data2['date'].dt.month
data2['day'] = data2['date'].dt.day

In [15]:
data2

,date,symbol,adj_close,volume,year,month,day
0,2010-01-04,MMM,43.783867,3640265.0,2010,1,4
1,2010-01-05,MMM,43.509628,3405012.0,2010,1,5
2,2010-01-06,MMM,44.126682,6301126.0,2010,1,6
3,2010-01-07,MMM,44.158325,5346240.0,2010,1,7
4,2010-01-08,MMM,44.469463,4073337.0,2010,1,8
...,...,...,...,...,...,...,...
1890269,2024-12-02,ZTS,176.809998,2391500.0,2024,12,2
1890270,2024-12-03,ZTS,176.940002,2679000.0,2024,12,3
1890271,2024-12-04,ZTS,175.320007,2687000.0,2024,12,4
1890272,2024-12-05,ZTS,174.770004,2442000.0,2024,12,5


In [16]:
data2.columns

Index(['date', 'symbol', 'adj_close', 'volume', 'year', 'month', 'day'], dtype='object')

In [17]:
cols = ['year', 'month', 'day', 'symbol', 'adj_close', 'volume']

In [18]:
data2 = data2[cols]

In [19]:
data2

,year,month,day,symbol,adj_close,volume
0,2010,1,4,MMM,43.783867,3640265.0
1,2010,1,5,MMM,43.509628,3405012.0
2,2010,1,6,MMM,44.126682,6301126.0
3,2010,1,7,MMM,44.158325,5346240.0
4,2010,1,8,MMM,44.469463,4073337.0
...,...,...,...,...,...,...
1890269,2024,12,2,ZTS,176.809998,2391500.0
1890270,2024,12,3,ZTS,176.940002,2679000.0
1890271,2024,12,4,ZTS,175.320007,2687000.0
1890272,2024,12,5,ZTS,174.770004,2442000.0


In [20]:
data2.dtypes

year           int32
month          int32
day            int32
symbol        object
adj_close    float64
volume       float64
dtype: object

In [38]:
df1 = data2.copy()

In [39]:
df1

,year,month,day,symbol,adj_close,volume
0,2010,1,4,MMM,43.783867,3640265.0
1,2010,1,5,MMM,43.509628,3405012.0
2,2010,1,6,MMM,44.126682,6301126.0
3,2010,1,7,MMM,44.158325,5346240.0
4,2010,1,8,MMM,44.469463,4073337.0
...,...,...,...,...,...,...
1890269,2024,12,2,ZTS,176.809998,2391500.0
1890270,2024,12,3,ZTS,176.940002,2679000.0
1890271,2024,12,4,ZTS,175.320007,2687000.0
1890272,2024,12,5,ZTS,174.770004,2442000.0


In [ ]:
df1[df1['symbol'] == 'AAPL' ]
df1[df1['year'] == 2023 ]
value_end = df1['adj_close'].iloc[-1]
value_in = df1['adj_close'].iloc[0]
print(value_end)
print(value_in)

In [44]:
df2 = data2.copy()

In [45]:
years = df2['year'].unique()
years

array([2010, 2011, 2012, 2013, 2014, 2015, 2016, 2017, 2018, 2019, 2020,
       2021, 2022, 2023, 2024], dtype=int32)

In [49]:
annual_return ={}
years = df2['year'].unique()
df2 = df2[df2['symbol'] == 'AAPL']
for year in years:
    df_filtered = df2[df2['year'] == year]
    value_end = df_filtered['adj_close'].iloc[-1]
    value_in = df_filtered['adj_close'].iloc[0]
    annual_return[year] = (value_end - value_in) / value_in 
print(annual_return)

{2010: 0.5072198098378983, 2011: 0.22887424391186606, 2012: 0.3055853501314697, 2013: 0.04750866187379936, 2014: 0.42628375002144264, 2015: -0.020822909266019388, 2016: 0.1238432283896102, 2017: 0.48042521479866884, 2018: -0.07054335355446169, 2019: 0.8874241458293838, 2020: 0.782399449982812, 2021: 0.38060956431544773, 2022: -0.28199507087785886, 2023: 0.5479823522511433, 2024: 0.3145317107950646}


In [ ]:
annual_df = pd.DataFrame.from_dict(annual_return)

ValueError: If using all scalar values, you must pass an index

In [ ]:
def cal_annual_return (df, symbol):
    annual_return ={}
    years = df['year'].unique()
    df = df[df['symbol'] == symbol]
    for year in years 
    value_end = df['adj_close'].iloc[-1]
    value_in = df['adj_close'].iloc[0]
    print(f'ValueEnd {value_end}')
    print(f'ValueIn {value_in}')
    
    return (value_end - value_in) / value_in 


In [41]:
prueba_apple = cal_annual_return(data2, 'AAPL', 2023)
prueba_apple

ValueEnd 191.5913848876953
ValueIn 123.7684555053711


0.5479823522511433

In [32]:
data2[(data2['symbol'] == 'AAPL') & (data2['year'] == 2023)]

,year,month,day,symbol,adj_close,volume
149834,2023,1,3,AAPL,123.768456,112117500.0
149835,2023,1,4,AAPL,125.045036,89113600.0
149836,2023,1,5,AAPL,123.718979,80962700.0
149837,2023,1,6,AAPL,128.271103,87754700.0
149838,2023,1,9,AAPL,128.795593,70790800.0
...,...,...,...,...,...,...
150079,2023,12,22,AAPL,192.656174,37122800.0
150080,2023,12,26,AAPL,192.108841,28919300.0
150081,2023,12,27,AAPL,192.208374,48087700.0
150082,2023,12,28,AAPL,192.636276,34049900.0


In [21]:
# Chat helped. 
annual_returns = data2.groupby(['symbol', 'year']).apply(lambda group: (group['adj_close'].iloc[-1] / group['adj_close'].iloc[0]) - 1).reset_index(name='annual_return')

/var/folders/bs/hzz4qdm57z9cct0t_9rdynfr0000gn/T/ipykernel_35455/1831142863.py:2: DeprecationWarning: DataFrameGroupBy.apply operated on the grouping columns. This behavior is deprecated, and in a future version of pandas the grouping columns will be excluded from the operation. Either pass `include_groups=False` to exclude the groupings or explicitly select the grouping columns after groupby to silence this warning.
  annual_returns = data2.groupby(['symbol', 'year']).apply(lambda group: (group['adj_close'].iloc[-1] / group['adj_close'].iloc[0]) - 1).reset_index(name='annual_return')


In [22]:
annual_returns[annual_returns['symbol'] == 'AAPL']

,symbol,year,annual_return
15,AAPL,2010,0.507220
16,AAPL,2011,0.228874
17,AAPL,2012,0.305585
18,AAPL,2013,0.047509
19,AAPL,2014,0.426284
20,AAPL,2015,-0.020823
21,AAPL,2016,0.123843
22,AAPL,2017,0.480425
23,AAPL,2018,-0.070543
24,AAPL,2019,0.887424


In [23]:
annual_returns

,symbol,year,annual_return
0,A,2010,0.323642
1,A,2011,-0.165950
2,A,2012,0.133482
3,A,2013,0.379639
4,A,2014,0.025399
...,...,...,...
7170,ZTS,2020,0.240854
7171,ZTS,2021,0.499937
7172,ZTS,2022,-0.369204
7173,ZTS,2023,0.356284


In [ ]:
data[data['Symbol'] == 'AAPL']

In [ ]:
data['Symbol'] == ''

In [ ]:
data2

In [ ]:
data2.head(12)

In [ ]:
data3

In [ ]:
data2 = data.copy()

In [24]:
df

,exchange,symbol,shortname,longname,sector,industry,currentprice,marketcap,ebitda,revenuegrowth,city,state,country,fulltimeemployees,longbusinesssummary,weight
0,NMS,AAPL,Apple Inc.,Apple Inc.,Technology,Consumer Electronics,242.84,3670720643072,1.346610e+11,0.061,Cupertino,CA,United States,164000.0,"Apple Inc. designs, manufactures, and markets ...",0.064589
1,NMS,NVDA,NVIDIA Corporation,NVIDIA Corporation,Technology,Semiconductors,142.44,3488355713024,6.118400e+10,1.224,Santa Clara,CA,United States,29600.0,NVIDIA Corporation provides graphics and compu...,0.061381
2,NMS,MSFT,Microsoft Corporation,Microsoft Corporation,Technology,Software - Infrastructure,443.57,3297889746944,1.365520e+11,0.160,Redmond,WA,United States,228000.0,Microsoft Corporation develops and supports so...,0.058029
3,NMS,AMZN,"Amazon.com, Inc.","Amazon.com, Inc.",Consumer Cyclical,Internet Retail,227.03,2387220627456,1.115830e+11,0.110,Seattle,WA,United States,1551000.0,"Amazon.com, Inc. engages in the retail sale of...",0.042005
4,NMS,GOOG,Alphabet Inc.,Alphabet Inc.,Communication Services,Internet Content & Information,176.49,2149065949184,1.234700e+11,0.151,Mountain View,CA,United States,181269.0,Alphabet Inc. offers various products and plat...,0.037815
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
498,NYQ,BWA,BorgWarner Inc.,BorgWarner Inc.,Consumer Cyclical,Auto Parts,33.80,7392059904,1.882000e+09,-0.048,Auburn Hills,MI,United States,39900.0,"BorgWarner Inc., together with its subsidiarie...",0.000130
499,NYQ,HII,"Huntington Ingalls Industries,","Huntington Ingalls Industries, Inc.",Industrials,Aerospace & Defense,187.95,7354351616,1.071000e+09,-0.024,Newport News,VA,United States,44000.0,"Huntington Ingalls Industries, Inc. designs, b...",0.000129
500,NYQ,FMC,FMC Corporation,FMC Corporation,Basic Materials,Agricultural Inputs,56.58,7063221248,7.033000e+08,0.085,Philadelphia,PA,United States,5800.0,"FMC Corporation, an agricultural sciences comp...",0.000124
501,NMS,QRVO,"Qorvo, Inc.","Qorvo, Inc.",Technology,Semiconductors,68.33,6459010048,6.731300e+08,-0.052,Greensboro,NC,United States,8700.0,"Qorvo, Inc. engages in development and commerc...",0.000114


In [43]:
df.head(20)

,exchange,symbol,shortname,longname,sector,industry,currentprice,marketcap,ebitda,revenuegrowth,city,state,country,fulltimeemployees,longbusinesssummary,weight
0,NMS,AAPL,Apple Inc.,Apple Inc.,Technology,Consumer Electronics,242.84,3670720643072,1.346610e+11,0.061,Cupertino,CA,United States,164000.0,"Apple Inc. designs, manufactures, and markets ...",0.064589
1,NMS,NVDA,NVIDIA Corporation,NVIDIA Corporation,Technology,Semiconductors,142.44,3488355713024,6.118400e+10,1.224,Santa Clara,CA,United States,29600.0,NVIDIA Corporation provides graphics and compu...,0.061381
2,NMS,MSFT,Microsoft Corporation,Microsoft Corporation,Technology,Software - Infrastructure,443.57,3297889746944,1.365520e+11,0.160,Redmond,WA,United States,228000.0,Microsoft Corporation develops and supports so...,0.058029
3,NMS,AMZN,"Amazon.com, Inc.","Amazon.com, Inc.",Consumer Cyclical,Internet Retail,227.03,2387220627456,1.115830e+11,0.110,Seattle,WA,United States,1551000.0,"Amazon.com, Inc. engages in the retail sale of...",0.042005
4,NMS,GOOG,Alphabet Inc.,Alphabet Inc.,Communication Services,Internet Content & Information,176.49,2149065949184,1.234700e+11,0.151,Mountain View,CA,United States,181269.0,Alphabet Inc. offers various products and plat...,0.037815
5,NMS,GOOGL,Alphabet Inc.,Alphabet Inc.,Communication Services,Internet Content & Information,174.71,2147972284416,1.234700e+11,0.151,Mountain View,CA,United States,181269.0,Alphabet Inc. offers various products and plat...,0.037795
6,NMS,META,"Meta Platforms, Inc.","Meta Platforms, Inc.",Communication Services,Internet Content & Information,623.77,1574701105152,7.920900e+10,0.189,Menlo Park,CA,United States,72404.0,"Meta Platforms, Inc. engages in the developmen...",0.027708
7,NMS,TSLA,"Tesla, Inc.","Tesla, Inc.",Consumer Cyclical,Auto Manufacturers,389.22,1249419591680,1.324400e+10,0.078,Austin,TX,United States,140473.0,"Tesla, Inc. designs, develops, manufactures, l...",0.021985
8,NYQ,BRK-B,Berkshire Hathaway Inc. New,Berkshire Hathaway Inc.,Financial Services,Insurance - Diversified,470.50,1013791064064,1.495470e+11,-0.002,Omaha,NE,United States,396500.0,"Berkshire Hathaway Inc., through its subsidiar...",0.017839
9,NMS,AVGO,Broadcom Inc.,Broadcom Inc.,Technology,Semiconductors,179.53,838509264896,2.295800e+10,0.164,Palo Alto,CA,United States,20000.0,"Broadcom Inc. designs, develops, and supplies ...",0.014754
